## Search Engine with Tools and Agents

In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

### Prebuilt Tools

In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv, )
arxiv.name

'arxiv'

In [4]:
tools = [wiki, arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Learning\\Generative_AI\\ChatBot\\myenv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

### Custom Tools [RAG Tool]

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["HUGGING_FACE"] = os.getenv('HUGGING_FACE')

embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

c:\Learning\Generative_AI\ChatBot\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Learning\Generative_AI\ChatBot\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:

docs = WebBaseLoader('https://en.wikipedia.org/wiki/Voyager_1').load()
splitted_docs = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorestore = FAISS.from_documents(splitted_docs, embedding)
retriever = vectorestore.as_retriever()

In [8]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001EAE2BD0340>)

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, 'voyeger_search', "search any information about voyeger")
retriever_tool.name

'voyeger_search'

In [10]:
tools = [wiki, arxiv, retriever_tool]

In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Learning\\Generative_AI\\ChatBot\\myenv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='voyeger_search', description='search any information about voyeger', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001EA9445EEF0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001EAE2BD

### Running all these Tools with Agents and LLM Models
`Toola, LLM --> AgentExecutor`

In [12]:
from langchain_groq import ChatGroq
groq_api = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model='gemma2-9b-it', api_key=groq_api)

In [13]:
## Pre-built Prompt
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

c:\Learning\Generative_AI\ChatBot\myenv\lib\site-packages\langsmith\client.py:5434: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [14]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
## Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_has

In [16]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
agent_executer.invoke({"input": "Tell me about voyeger 1"})



> Entering new AgentExecutor chain...

Invoking: `voyeger_search` with `{'query': 'voyager 1'}`


Retrieved from "https://en.wikipedia.org/w/index.php?title=Voyager_1&oldid=1241201888"

^ a b c Peat, Chris (September 9, 2012). "Spacecraft escaping the Solar System". Heavens-Above. Archived from the original on May 11, 2018. Retrieved March 16, 2014.

^ a b Wolchover, Natalie (October 9, 2012). "Did NASA's Voyager 1 Spacecraft Just Exit the Solar System?". livescience. Archived from the original on October 3, 2013. Retrieved August 20, 2013.

^ Matson, John (December 4, 2012). "Despite Tantalizing Hints, Voyager 1 Has Not Crossed into the Interstellar Medium". Scientific American. Archived from the original on March 13, 2013. Retrieved August 20, 2013.

^ "Voyager 1 Can 'Taste' the Interstellar Shore". Discovery News. Discovery Channel. December 3, 2012. Archived from the original on December 5, 2012. Retrieved September 16, 2013.

^ Oakes, Kelly (December 3, 2012). "Voyager 1 is stil

{'input': 'Tell me about voyeger 1',
 'output': 'Voyager 1 launched on September 5, 1977, and is the first spacecraft to enter interstellar space.'}

In [18]:
agent_executer.invoke({"input": "tell me about this research paper: 1706.03762"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.003762'}`


No good Arxiv Result was foundI am sorry, I don't have access to information about research papers not found on Arxiv.

> Finished chain.


{'input': 'tell me about this research paper: 1706.03762',
 'output': "I am sorry, I don't have access to information about research papers not found on Arxiv."}